In [1]:
import os
import csv
import sqlite3
import datetime
import pickle

UNICODE_ENCODING = 'utf-8'
NO_DATA_NUMERIC = None
NO_DATA_UNICODE = None
GENERATION_FILE = "generation_totals.csv"
OUTPUT_DIR = "./output_directory"

class PowerPlant:
    def __init__(self, country, capacity, primary_fuel, generation=None, idnr=None):
        self.country = country
        self.capacity = capacity
        self.primary_fuel = primary_fuel
        self.generation = generation
        self.idnr = idnr
        self.estimated_generation_gwh = NO_DATA_NUMERIC

class Location:
    def __init__(self, latitude, longitude):
        self.latitude = latitude
        self.longitude = longitude

# String cleaning function
def format_string(value, encoding=UNICODE_ENCODING):
    return value.replace("\n", " ").replace("\r", " ").replace(",", " ").strip().replace(u"\u001A", "")

# Read CSV data into dictionary
def read_csv_file_to_dict(filename):
    print("Reading CSV file from:", filename)
    with open(filename, 'r', encoding='utf-8') as fin:
        reader = csv.DictReader(fin)
        pdb = {}  # returned object
        for row in reader:
            row = {k: format_string(v) for k, v in row.items()}
            try:
                row['capacity_mw'] = float(row['capacity_mw'])
            except:
                row['capacity_mw'] = None
            # check if primary fuel is an empty string
            if not row['primary_fuel']:
                row['primary_fuel'] = None
            pdb[row['gppd_idnr']] = row
        print("CSV data read successfully.")
        return pdb

# Predict primary fuel based on frequency
def predict_primary_fuel(data):
    fuels = [plant['primary_fuel'] for plant in data.values() if plant['primary_fuel'] is not None]
    if fuels:
        return max(set(fuels), key=fuels.count)
    else:
        return "Unknown"

# Predict average capacity in megawatts
def predict_capacity_mw(data):
    capacities = [plant['capacity_mw'] for plant in data.values() if plant['capacity_mw'] is not None]
    return sum(capacities) / len(capacities) if capacities else 0

# Main function
def main():
    # Local file path
    file_path = r"C:\Users\Zaina\Downloads\database_IND.csv"

    # Read the CSV file into a dictionary
    data = read_csv_file_to_dict(file_path)

    # Perform predictions
    predicted_primary_fuel = predict_primary_fuel(data)
    predicted_capacity_mw = predict_capacity_mw(data)

    # Output predictions
    print("Predicted Primary Fuel:", predicted_primary_fuel)
    print("Predicted Capacity (MW):", predicted_capacity_mw)

if __name__ == "__main__":
    main()

Reading CSV file from: C:\Users\Zaina\Downloads\database_IND.csv
CSV data read successfully.
Predicted Primary Fuel: Coal
Predicted Capacity (MW): 326.2237552370453
